In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
import torch

print("PyTorch Version: ", torch.__version__)
print("CUDA Available: ", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA Version: ", torch.version.cuda)
    print("cuDNN Version: ", torch.backends.cudnn.version())
    print("GPU: ", torch.cuda.get_device_name(0))

PyTorch Version:  2.6.0+cpu
CUDA Available:  False


In [4]:
import tensorflow as tf
import transformers

print(tf.__version__)  
print(transformers.__version__) 


c:\Users\edayd\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.18.0
4.49.0


In [5]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from transformers import AutoModel, AutoTokenizer

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

# Check if CUDA is available and move the model to GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Assuming dataset is loaded as a DatasetDict with a single split, access the single dataset directly
dataset = load_dataset('csv', data_files="dataset.csv")['train']

num_labels = len(set(dataset['label']))  # Assuming 'labels' is your label column
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=num_labels)

# Modify the train_test_split call to include a validation split
split_datasets = dataset.train_test_split(test_size=0.2)  # Split 20% of the data for testing/validation
validation_split = split_datasets['test'].train_test_split(test_size=0.5)  # Split the test set into half for validation

# Now, merge the new validation set back into split_datasets under the correct keys
split_datasets['validation'] = validation_split['test']
split_datasets['test'] = validation_split['train']

# Continue with tokenization and the rest of your code
def tokenize_function(examples):
	return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = split_datasets.map(tokenize_function, batched=True)
'''
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',
)'''


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,              # Daha az epoch, overfitting önlemek için
    per_device_train_batch_size=32,  # GPU belleğine bağlı olarak artırılabilir
    per_device_eval_batch_size=64,   # Değerlendirme sırasında daha büyük batch size
    learning_rate=3e-5,              # Daha küçük öğrenme oranı
    warmup_steps=2000,               # Daha uzun ısınma süresi
    weight_decay=0.05,               # Overfitting’i önlemek için
    logging_dir='./logs',
    logging_steps=2000,              # Log kaydetme sıklığını düşürdük
    save_steps=5000,                 # Modeli daha seyrek kaydediyoruz
    save_total_limit=3,              # En iyi 3 modeli kaydet
    evaluation_strategy="epoch",     # Her epoch sonunda değerlendirme yap
    fp16=True                        # Eğer GPU destekliyorsa eğitimi hızlandırır
)

# Adjust the Trainer initialization to use the correct keys
trainer = Trainer(
	model=model,
	args=training_args,
	train_dataset=tokenized_datasets['train'],
	eval_dataset=tokenized_datasets['validation']  # Use 'validation' instead of 'test'
)

# The rest of your code remains the same

# Step 5: Train
trainer.train()

# Step 6: Save the model
model.save_pretrained('./model2')


trainer.evaluate()

In [ ]:
label_map = {
    'VETERİNER İŞLERİ MÜDÜRLÜĞÜ': 0, 'SOSYAL DESTEK HİZMETLERİ MÜDÜRLÜĞÜ': 1, 'FEN İŞLERİ MÜDÜRLÜĞÜ': 2, 'PARK VE BAHÇELER MÜDÜRLÜĞÜ': 3, 'ZABITA MÜDÜRLÜĞÜ': 4, 'YAPI KONTROL MÜDÜRLÜĞÜ': 5, 'TEMİZLİK İŞLERİ MÜDÜRLÜĞÜ': 6, 'BİLGİ İŞLEM MÜDÜRLÜĞÜ': 7, 'DESTEK HİZMETLERİ MÜDÜRLÜĞÜ': 8, 'İKLİM DEĞİŞİKLİĞİ VE SIFIR ATIK MÜDÜRLÜĞÜ': 9, 'ÖZEL KALEM MÜDÜRLÜĞÜ': 10, 'RUHSAT DENETİM MÜDÜRLÜĞÜ': 11, 'İNSAN KAYNAKLARI VE EĞİTİM MÜDÜRLÜĞÜ': 12, 'GENÇLİK VE SPOR HİZMETLERİ MÜDÜRLÜĞÜ': 13, 'BASIN YAYIN VE HALKLA İLİŞKİLER MÜDÜRLÜĞÜ': 14, 'MALİ HİZMETLER MÜDÜRLÜĞÜ': 15, 'STRATEJİ GELİŞTİRME MÜDÜRLÜĞÜ': 16, 'İMAR VE ŞEHİRCİLİK MÜDÜRLÜĞÜ': 17, 'PLAN VE PROJE MÜDÜRLÜĞÜ': 18, 'KENTSEL DÖNÜŞÜM MÜDÜRLÜĞÜ': 19, 'KÜLTÜR İŞLERİ MÜDÜRLÜĞÜ': 20, 'EMLAK VE İSTİMLAK MÜDÜRLÜĞÜ': 21, 'YAZI İŞLERİ MÜDÜRLÜĞÜ': 22, 'SAĞLIK İŞLERİ MÜDÜRLÜĞÜ': 23, 'AFET İŞLERİ MÜDÜRLÜĞÜ': 24, 'İŞLETME MÜDÜRLÜĞÜ': 25
}

In [12]:
from transformers import AutoTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax


model_path = './model2'
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForSequenceClassification.from_pretrained(model_path)

# hayvan ölüsü var gelin burdan alın kötü
text = "beli̇rti̇len adreste bulunan gi̇ysi̇ kumbarasinin i̇çeri̇si̇ne vatandaş yanlişlikla nüfus cüzdanini düşürdüğü beli̇rti̇lerek vatandaş yardim talep eder  "


inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)


model.eval()

with torch.no_grad():
	outputs = model(**inputs)


logits = outputs.logits

# vektöre dönüştürmeks
scores = softmax(logits, dim=1)

# ilk 3 ünü getittiriyorum
num_labels = model.config.num_labels

top_scores, top_indices = torch.topk(scores, 3)


for i in range(3):
    label_index = top_indices[0][i].item()
    label_description = label_map.get(label_index, "Unknown Label")
    print(f"Score for label '{label_description}': {top_scores[0][i].item()}")

Score for label 'İKLİM DEĞİŞİKLİĞİ VE SIFIR ATIK MÜDÜRLÜĞÜ': 0.7512940168380737
Score for label 'TEMİZLİK İŞLERİ MÜDÜRLÜĞÜ': 0.22584660351276398
Score for label 'SÜPER HİZMET': 0.006471273489296436
